# Evo-Bayes-Agent

In [1]:
from Ionomy import IonPanda, BitTA
from decouple import config

MARKET = 'usd-btc'
CURRENCY = 'btc'
BASE = 'usd'
TIME = 'hour'
initial_capital = 5_000
MAX_BUY = 0.1
MAX_SELL = 0.1
FEE_RATE = 0.01

window_size = 32
layer_size = 500
output_size = 3

bta = BitTA(config('TREX_KEY'), config('TREX_SECRET'))
ionpd = IonPanda(config('IONOMY_KEY'), config('IONOMY_SECRET'))
bta.update(CURRENCY, BASE, TIME)
ohlcv_df = bta.df

In [2]:
from neuroevolution import DemiChad

Using TensorFlow backend.


In [3]:
demichad = DemiChad(ohlcv_df)

In [4]:
# demichad.meta_evolve()

In [ ]:
demichad.evolve(10_000)

gen	nevals	f_avg	f_std	f_min	f_max	l_avg  	l_std    	l_min	l_max 
0  	100   	-1   	0    	-1   	-1   	0.01786	0.029782 	0    	0.101 
1  	67    	-1   	0    	-1   	-1   	0.05037	0.0417586	0    	0.1015
2  	60    	-1   	0    	-1   	-1   	0.06735	0.0453228	0    	0.1015
3  	71    	-1   	0    	-1   	-1   	0.057095	0.0482611	0    	0.1035
4  	67    	-1   	0    	-1   	-1   	0.06565 	0.0463058	0    	0.1035
5  	64    	-1   	0    	-1   	-1   	0.06192 	0.0462507	0    	0.1035
6  	59    	-1   	0    	-1   	-1   	0.06406 	0.0473306	0    	0.108 
7  	68    	-1   	0    	-1   	-1   	0.0559  	0.050152 	0    	0.108 
8  	51    	-1   	0    	-1   	-1   	0.064975	0.0502491	0    	0.108 
9  	66    	-1   	0    	-1   	-1   	0.05676 	0.0517578	0    	0.108 
10 	72    	-1   	0    	-1   	-1   	0.042745	0.0493379	0    	0.108 
11 	62    	-1   	0    	-1   	-1   	0.059865	0.0520034	0    	0.108 
12 	75    	-1   	0    	-1   	-1   	0.063765	0.0510147	0    	0.108 
13 	67    	-1   	0    	-1   	-1   	0.05807 	0.0510036	0    	0.108 

In [ ]:
demichad.omega()

In [ ]:
demichad.plot()

In [ ]:
demichad.env.length

In [ ]:
buy_history

In [ ]:
sell_history